# Big Data Grab

A big grab of all the data to date.

In [1]:
from dakar_rallydj.getter import DakarAPIClient
           

def custom_key_fn(request, ignored_parameters=None, match_headers=False, serializer=None, **request_kwargs):
    return request.url.split("api/")[-1]  # Store the plain URL as the key

dakar = DakarAPIClient(
    use_cache=True,
    backend='sqlite',
    cache_name='dakar_cache_2025',
    expire_after=-1,  # Never expire
    key_fn=custom_key_fn
)

In [2]:
all_clazz = ["A", "F", "K", "M"]
all_stages = range(1, 13)

dakar.get_category()
dakar.get_groups()
dakar.get_clazz(category=all_clazz)
dakar.get_withdrawals(category=["A", "K", "M"])

for c in ["A", "M"]:
    dakar.get_stages(category=c)
    for s in all_stages:
        dakar.get_waypoints(category=c, stage=s)
        dakar.get_scores(category=c, stage=s)

In [3]:
from sqlite_utils import Database

db = Database("dakar_cache_2025.sqlite")  # Custom key function to use raw URLs
   
db.table_names()

['responses', 'redirects']

In [ ]:
# The responses are hashed URLs
for i, row in enumerate(db["responses"].rows):
    print(row)
    if i>10:
        break